In [ ]:
import subprocess
import cv2
from subprocess import check_output
import numpy as np
from scipy import spatial

Code structure-

Read Book:
    Convert PDF to Images
    Show page 1 (with transformation)
    while (!stopReading) :
        pageChange = Read Output Sensor
        if pageChange == nextPage :
            Read Page(+1)
        else if pageChange == prevPage :
            Read Page(-1)

In [ ]:
def get_num_pages(pdf_path):
    output = check_output(["pdfinfo", pdf_path]).decode()
    pages_line = [line for line in output.splitlines() if "Pages:" in line][0]
    num_pages = int(pages_line.split(":")[1])
    return num_pages

In [ ]:
def ConvertPdfToImages (pdfFile, imageFile) :
    
    # Path to Poppler's libraries (Download from https://blog.alivate.com.au/poppler-windows/)
    pdfToPPMPath = r"C:\Program Files (x86)\Poppler\poppler-0.68.0\bin\pdftoppm.exe"

    # Convert PDF to images
    subprocess.Popen('"%s" -png %s %s' % (pdfToPPMPath, pdfFile, imageFile))
    
    # Return number of pages in PDF
    return get_num_pages(pdfFile)

In [ ]:
def ReadPage (imageFile,pageNumber) :
    image = imageFile + '-' + str(pageNumber) + '.png'
    # Open CV code for transformation
    

In [ ]:
def ReadSensorOutput () :
    # Return 1 for next page, -1 for prev page based on threshold

In [ ]:
def ReadBook (pdfFile) :
    imageFile = pdfFile[:-4] # Prefix of image is PDF file name
    totalPages = ConvertPdfToImages(pdfFile, imageFile)
    pageNumber = 1
    ReadPage(imageFile,pageNumber)
    while (!stopReading) : # Replace with a listener to signal end of reading
        pageChange = ReadSensorOutput()
        if pageChange==1 and pageNumber<totalPages : 
            ReadPage(imageFile,pageNumber+1)
        elif pageChange==-1 and pageNumber>0 :
            ReadPage(imageFile,pageNumber-1)

In [ ]:
pdfFile = "258report.pdf"
ReadBook(pdfFile)